## ライブラリのインポート

In [9]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import requests


## セイコーマートの位置情報データの取得

### Overpass APIを使用してコンビニのデータを取得
Overpass APIを使用して、札幌市内のコンビニの位置データを取得します。

In [10]:

# Overpass APIエンドポイント
overpass_url = "http://overpass-api.de/api/interpreter"

# Overpass QLクエリ（札幌市内のコンビニ）
overpass_query = """
[out:json];
area[name="札幌市"]->.searchArea;
(
  node["shop"="convenience"](area.searchArea);
);
out body;
>;
out skel qt;
"""

# APIリクエストを送信
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# ノードを抽出
nodes = data['elements']

# ノードをGeoDataFrameに変換
coords = []
names = []
branches = []

for node in nodes:
    if node['type'] == 'node':
        coords.append(Point(node['lon'], node['lat']))
        names.append(node['tags'].get('name', 'Unnamed'))
        branches.append(node['tags'].get('branch', ''))

gdf = gpd.GeoDataFrame({'name': names, 'branch':branches,'geometry': coords})

In [11]:
gdf

,name,branch,geometry
0,ファミリーマート,札幌センチュリーロイヤルホテル店,POINT (141.34839 43.06665)
1,ローソン,札幌北5条西六丁目店,POINT (141.34698 43.06676)
2,ローソン,札幌北4条西四丁目店,POINT (141.34928 43.06591)
3,セイコーマート,北3条店,POINT (141.35501 43.06586)
4,セブン-イレブン,札幌北37条東店,POINT (141.38984 43.10397)
...,...,...,...
702,セイコーマート,,POINT (141.26133 43.11517)
703,ファミリーマート,札幌南5条西20丁目店,POINT (141.32785 43.05190)
704,セイコーマート,,POINT (141.39042 43.02891)
705,ローソン,S北海道科学大学店,POINT (141.24682 43.13368)


### 「セイコーマート」を含む行を抽出
すべてのコンビニから「セイコーマート」のみを抽出します。

In [12]:

seicomart_gdf = gdf[gdf['name'].str.contains("セイコーマート")]
seicomart_gdf

,name,branch,geometry
3,セイコーマート,北3条店,POINT (141.35501 43.06586)
50,セイコーマート,北9条店,POINT (141.35071 43.07165)
51,セイコーマート,つぼた店,POINT (141.34782 43.07012)
52,セイコーマート,北海道庁店,POINT (141.34664 43.06449)
53,セイコーマート,道庁前北店,POINT (141.34692 43.06520)
...,...,...,...
686,セイコーマート,狸小路2丁目店,POINT (141.35542 43.05773)
692,セイコーマート,札大正門前店,POINT (141.38731 43.00982)
702,セイコーマート,,POINT (141.26133 43.11517)
704,セイコーマート,,POINT (141.39042 43.02891)


HPによると札幌市内には、330件がある。
[店舗検索 | セイコーマート](https://store.seicomart.co.jp/mapsearch.php)

OSMはみんなでオープンデータの地理情報を作るプロジェクト。   
すべてのセイコーマートが登録されているわけではない。

### GISデータで出力

In [13]:
# seicomart_gdfをGeoPackageに出力
output_file = "output/seicomart.gpkg"
seicomart_gdf.to_file(output_file, driver="GPKG")

### 到達圏解析のために緯度経度の配列を作成

In [14]:
# seicomart_gdfから緯度経度の配列を取得
seicomart_coords = seicomart_gdf['geometry'].apply(lambda p: (p.y, p.x)).tolist()
seicomart_coords

[(43.0658618, 141.3550123),
 (43.0716453, 141.3507115),
 (43.0701156, 141.3478226),
 (43.0644919, 141.3466425),
 (43.0652016, 141.346922),
 (43.0646385, 141.3497592),
 (43.0646933, 141.3553874),
 (43.0616387, 141.3524414),
 (43.0580407, 141.3441828),
 (43.0731229, 141.3523229),
 (43.0557871, 141.3522918),
 (43.0605514, 141.3576188),
 (43.0607504, 141.3641057),
 (43.0488981, 141.3521192),
 (43.0484318, 141.3501356),
 (43.056676, 141.3301704),
 (42.9368175, 141.3426568),
 (43.0645692, 141.2581319),
 (43.0633285, 141.3306812),
 (43.079503, 141.306629),
 (43.0452104, 141.3568574),
 (43.05109, 141.3542604),
 (43.0730923, 141.299638),
 (43.080593, 141.28684),
 (43.0752097, 141.3482015),
 (43.131317, 141.20316),
 (43.0504804, 141.2985338),
 (43.0668265, 141.332558),
 (43.0727328, 141.3235826),
 (43.0589965, 141.3280447),
 (43.0626383, 141.3220661),
 (43.0656372, 141.3190625),
 (43.087611, 141.267536),
 (43.084927, 141.278394),
 (43.0623246, 141.3140477),
 (43.029696, 141.434728),
 (43.0671429

## セイコーマートの到達圏解析

### グラフの準備
今回は札幌市内のセイコーマートの到達圏解析を実施したいため、OSMNXを使用して、札幌市内のグラフデータ（道路のデータ）を取得する。

In [15]:

# ネットワークグラフを取得
place_name = "Sapporo, Japan"
G = ox.graph_from_place(place_name, network_type='walk')

In [16]:
# GraphをGeoPackageに出力
ox.save_graph_geopackage(G, filepath='output/graph.gpkg')

### 到達圏解析
グラフデータと、セイコーマートの緯度経度をもとに350mの範囲の到達圏解析を実施する

> 商圏は常に一定なわけではありません。たとえば自分がコンビニに行く場合と、大型スーパーに行く場合を考えてみるとわかります。家の近所に店がない場合、多少離れたスーパーへは行くとしても、わざわざ遠くのコンビニまで足を延ばすことは少ないでしょう。コンビニでの買い物は会社の帰りなど、“ついで”に済ませるのではないでしょうか。このように、小売業の場合、一般的に店舗の規模によって商圏の大きさが変わると言われています。コンビニならば徒歩2~5分（おおよそ350m圏）程度、大型スーパーならば徒歩、自転車、車などの移動時間が数十分程度の範囲が商圏になります。
https://www.zenrin-ms.co.jp/blog/2019/005/

In [17]:
all_polygons = []
all_edges = []
meters = 350 # 徒歩範囲

# セイコーマートの座標ごとに処理
for coord in seicomart_coords:
    # 最寄りのノードを取得し、到達圏の半径でサブグラフを作成
    nearest_node = ox.distance.nearest_nodes(G, coord[1], coord[0])
    subgraph = nx.ego_graph(G, nearest_node, radius=meters, distance='length')
    
    # サブグラフをもとに、到達圏のポリゴンとエッジを作成
    nodes, edges = ox.graph_to_gdfs(subgraph)
    convex_hull = nodes.unary_union.convex_hull
    all_polygons.append({'geometry': convex_hull})
    all_edges.append(edges[['geometry']])

# GISデータに出力
polygons_gdf = gpd.GeoDataFrame(all_polygons, crs="EPSG:4326")
edges_gdf = gpd.GeoDataFrame(pd.concat(all_edges, ignore_index=True), crs="EPSG:4326")
polygon_output_file = "output/seicomart_350m_walk_polygons.gpkg"
edges_output_file = "output/seicomart_350m_walk_edges.gpkg"

polygons_gdf.to_file(polygon_output_file, driver="GPKG")
edges_gdf.to_file(edges_output_file, driver="GPKG")